In [1]:
import tensorflow as tf
import numpy as np

# Logic gate definition

In [2]:
gate_and  = lambda x: x[0] and x[1]
gate_or   = lambda x: x[0] or  x[1]
gate_nand = lambda x: 1 - gate_and(x)
gate_nor  = lambda x: 1 - gate_or(x)
gate_xor  = lambda x: gate_and((gate_or(x), gate_nand(x)))

# Experiment

In [3]:
def nn_experiment(nn):
    gate_list = [('AND', gate_and), ('OR', gate_or), ('NAND', gate_nand), ('NOR', gate_nor), ('XOR', gate_xor)]
    data_x = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]]).astype(np.float32)
    results = [['X1', 'X2']] + [['%d'%data_x[i][0], '%d'%data_x[i][1]] for i in xrange(4)]
    
    for gate_name, gate_body in gate_list:
        print('Training %s gate ...'%(gate_name))
        
        # Generate gold-standard answers
        data_y = np.array(map(gate_body, [(_[0], _[1]) for _ in data_x])).reshape(4, 1)
        
        # Fit the neural network
        optimizer = tf.train.GradientDescentOptimizer(tf.Variable(0.1))
        train_op = optimizer.minimize(nn.loss)
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            num_iter = 20000
            for i in xrange(num_iter):
                sess.run(train_op, feed_dict={nn.X:data_x, nn.y:data_y})
            
            # Store results
            prediction = np.round(sess.run(nn.hypothesis, feed_dict={nn.X:data_x}))
            results[0].append(gate_name)
            for i in xrange(4):
                results[i+1].append('%d'%prediction[i][0])
    
    # Print results
    print('')
    for row in results:
        print('\t'.join(row))

# Simple gate NN

In [4]:
class SimpleGate(object):
    def __init__(self):
        self.X  = tf.placeholder(tf.float32)
        self.y  = tf.placeholder(tf.float32)
        self.W = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))        
        self.b = tf.Variable(tf.zeros([1]))
        self.hypothesis = tf.sigmoid(tf.matmul(self.X, self.W) + self.b)
        self.loss = tf.reduce_mean(-self.y * tf.log(self.hypothesis) - (1-self.y) * tf.log(1-self.hypothesis))

nn_experiment(SimpleGate())

Training AND gate ...
Training OR gate ...
Training NAND gate ...
Training NOR gate ...
Training XOR gate ...

X1	X2	AND	OR	NAND	NOR	XOR
0	0	0	0	1	1	0
0	1	0	1	1	0	0
1	0	0	1	1	0	0
1	1	1	1	0	0	0


# Deep gate NN

In [5]:
class DeepGate(object):
    def __init__(self):
        self.X = tf.placeholder(tf.float32)
        self.y = tf.placeholder(tf.float32)
        self.W1 = tf.Variable(tf.random_uniform([2, 5], -1.0, 1.0))
        self.W2 = tf.Variable(tf.random_uniform([5, 4], -1.0, 1.0))
        self.W3 = tf.Variable(tf.random_uniform([4, 1], -1.0, 1.0))        
        self.b1 = tf.Variable(tf.zeros([5]))
        self.b2 = tf.Variable(tf.zeros([4]))
        self.b3 = tf.Variable(tf.zeros([1]))
        self.L2 = tf.sigmoid(tf.matmul(self.X,  self.W1) + self.b1)
        self.L3 = tf.sigmoid(tf.matmul(self.L2, self.W2) + self.b2)        
        self.hypothesis = tf.sigmoid(tf.matmul(self.L3, self.W3) + self.b3)
        self.loss = tf.reduce_mean(-self.y * tf.log(self.hypothesis) - (1-self.y) * tf.log(1-self.hypothesis))
        
nn_experiment(DeepGate())

Training AND gate ...
Training OR gate ...
Training NAND gate ...
Training NOR gate ...
Training XOR gate ...

X1	X2	AND	OR	NAND	NOR	XOR
0	0	0	0	1	1	0
0	1	0	1	1	0	1
1	0	0	1	1	0	1
1	1	1	1	0	0	0
